In [1]:
import os
import sys
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
import platform
import numpy as np

In [3]:
import tensorflow as tf
import tensorflow.keras
from keras.preprocessing.image import ImageDataGenerator

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils  import plot_model

In [5]:
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications import VGG16

In [6]:
print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")

Python Platform: Linux-5.19.0-45-generic-x86_64-with-glibc2.35
Tensor Flow Version: 2.12.1
Keras Version: 2.12.0

Python 3.8.19 (default, Apr  6 2024, 17:58:10) 
[GCC 11.4.0]


In [8]:
tf.config.experimental.list_physical_devices('GPU')

[]

In [ ]:
from FairFaceDataset import *
fair_face = FairFace(
    under_sample = True,
    overwrite_sample_number = 100
)

In [ ]:
BATCH_SIZE = 64

In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split = 0.2
)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    fair_face.train_df,
    target_size = (fair_face.IMAGE_WIDTH, fair_face.IMAGE_HEIGHT),
    x_col = fair_face.X_COL,
    y_col = fair_face.Y_COL, 
    batch_size = BATCH_SIZE,
    class_mode = "categorical",
    subset = "training"
)

In [ ]:
validation_generator = train_datagen.flow_from_dataframe(
    fair_face.train_df,
    target_size = (fair_face.IMAGE_WIDTH, fair_face.IMAGE_HEIGHT),
    x_col = fair_face.X_COL,
    y_col = fair_face.Y_COL,
    batch_size = BATCH_SIZE,
    class_mode = "categorical",
    subset = "validation"
)

In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
val_generator = val_datagen.flow_from_dataframe(
    fair_face.test_df[:100],
    target_size = (fair_face.IMAGE_WIDTH, fair_face.IMAGE_HEIGHT),
    x_col = fair_face.X_COL,
    y_col = fair_face.Y_COL, 
    batch_size = BATCH_SIZE,
    class_mode = "categorical"
)

In [ ]:
base_vgg16 = VGG16(
    include_top = True,
    weights = None,
    input_shape = (fair_face.IMAGE_WIDTH, fair_face.IMAGE_HEIGHT, fair_face.NUM_CHANNELS),
    classes = 2
)

In [ ]:
cnn1 = Sequential()

cnn1.add(
    Conv2D(
        input_shape = (fair_face.IMAGE_WIDTH, fair_face.IMAGE_HEIGHT, fair_face.NUM_CHANNELS),
        filters=num_filters, 
        kernel_size=(3,3), 
        padding='same')
)
cnn1.add( Activation('relu'))
cnn1.add( MaxPooling2D(pool_size=(2, 2)))
    

# add one layer on flattened output
cnn1.add( Flatten() )
cnn1.add( Dense(100, activation='relu') )
cnn1.add( Dense(100, activation='relu') )
cnn1.add( Dense(2, activation='softmax') )

# Let's train the model 
cnn1.compile(loss='mean_squared_error',
              optimizer='rmsprop',
              metrics=['accuracy'])

print(cnn1.summary())

In [ ]:
history = cnn1.fit(
    train_generator,
    epochs=3,
    validation_data=val_generator,
    workers=10,
    use_multiprocessing=True
)